In [14]:
import json
from urllib.parse import urlparse

In [15]:
def extract_domains(data):
    """
    Extract unique source domains from article URLs in the data
    """
    domains = set()
    for item in data:
        url = item.get('article_url', '')
        if url:
            domain = urlparse(url).netloc
            # Remove 'www.' prefix if present
            domain = domain.replace('www.', '')
            domains.add(domain)
    
    return sorted(list(domains))

In [16]:
with open("data/public_test_acm.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'data/public_test_acm.json'

In [5]:
domains = extract_domains(data)

In [6]:
domains

['',
 'abc30.com',
 'arabnews.com',
 'arkansasonline.com',
 'bbc.com',
 'bloomberg.com',
 'cbc.ca',
 'chinese.uhrp.org',
 'cn.nytimes.com',
 'cnnphilippines.com',
 'commons.wikimedia.org',
 'cpr.org',
 'denverpost.com',
 'dvidshub.net',
 'edition.cnn.com',
 'en.wikipedia.org',
 'english.aawsat.com',
 'georgeherald.com',
 'gettyimages.de',
 'gettyimages.fi',
 'google.com',
 'irishnews.com',
 'lonelyplanet.com',
 'm.lasvegassun.com',
 'military.com',
 'mirror.co.uk',
 'mprnews.org',
 'nbcnews.com',
 'news.cgtn.com',
 'npr.org',
 'nytimes.com',
 'phys.org',
 'reuters.com',
 'snopes.com',
 'sputniknews.com',
 'theconversation.com',
 'theguardian.com',
 'theknow.denverpost.com',
 'thescottishsun.co.uk',
 'voanews.com',
 'washingtonpost.com',
 'wbur.org',
 'youwantedalist.com']

In [ ]:
# Import necessary libraries
import os
import json
import requests
import shutil
import imghdr
from PIL import Image
from tqdm import tqdm
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# The function you provided for downloading images
def download_and_save_image(image_url, save_folder_path, file_name):
    try:
        response = requests.get(image_url, stream=True, timeout=(60, 60))
        if response.status_code == 200:
            response.raw.decode_content = True
            image_path = os.path.join(save_folder_path, file_name)
            with open(image_path, 'wb') as f:
                shutil.copyfileobj(response.raw, f)
            if imghdr.what(image_path) and imghdr.what(image_path).lower() == 'png':
                img_fix = Image.open(image_path)
                img_fix.convert('RGB').save(image_path)
            return 1 
        else:
            print(f"Failed to download {image_url}: HTTP status {response.status_code}")
            return 0
    except Exception as e:
        print(f"Error downloading {image_url}: {str(e)}")
        return 0

# Function to get the filename from html_path and change extension to jpg
def get_image_filename(html_path):
    if not html_path:
        return None
    
    # Extract the base filename from the html_path
    base_name = os.path.basename(html_path)
    # Change extension to jpg
    if '.' in base_name:
        image_filename = os.path.splitext(base_name)[0] + '.jpg'
    else:
        image_filename = base_name + '.jpg'
    
    return image_filename

# Process a single inverse annotation file
def update_inverse_file(inverse_file_path):
    try:
        # Read the inverse annotation file
        with open(inverse_file_path, 'r', encoding='utf-8') as f:
            inverse_data = json.load(f)
        
        # Get the folder where the inverse file is located
        folder_path = os.path.dirname(inverse_file_path)
        
        # Track if we made any changes
        changes_made = False
        download_tasks = []
        
        # Process all_matched_captions
        if "all_matched_captions" in inverse_data:
            for i, caption in enumerate(inverse_data["all_matched_captions"]):
                if "image_link" in caption and "image_path" not in item:
                    # Get filename from html_path or use default
                    image_filename = get_image_filename(caption.get("html_path"))
                    
                    if not image_filename:
                        image_filename =  f"{i}.jpg"
                    
                    # Add to download tasks
                    download_tasks.append({
                        "url": caption["image_link"],
                        "folder": folder_path,
                        "filename": image_filename,
                        "entry": caption
                    })
        
        # Process matched_no_text
        if "matched_no_text" in inverse_data:
            for i, item in enumerate(inverse_data["matched_no_text"]):
                if "image_link" in item and "image_path" not in item:
                    # Get filename from html_path or use default
                    image_filename = get_image_filename(item.get("html_path")) or f"no_text_{i}.jpg"
                    
                    # Add to download tasks
                    download_tasks.append({
                        "url": item["image_link"],
                        "folder": folder_path,
                        "filename": image_filename,
                        "entry": item
                    })
        
        # Download images and update entries
        for task in download_tasks:
            success = download_and_save_image(task["url"], task["folder"], task["filename"])
            if success:
                task["entry"]["image_path"] = os.path.join(task["folder"], task["filename"])
                changes_made = True
        
        # Save the updated inverse file if changes were made
        if changes_made:
            with open(inverse_file_path, 'w', encoding='utf-8') as f:
                print(inverse_data)
                json.dump(inverse_data, f, indent=4, ensure_ascii=False)
            return True
        
        return False
    
    except Exception as e:
        logger.error(f"Error processing file {inverse_file_path}: {str(e)}")
        return False

# Find all folders in test directory
def get_test_folders(test_dir):
    return [os.path.join(test_dir, folder) for folder in os.listdir(test_dir) 
            if os.path.isdir(os.path.join(test_dir, folder))]

# Main execution cell
# Define the base test directory
test_dir = "queries_dataset/merged_balanced/inverse_search/test"

# Get all folders in the test directory
test_folders = get_test_folders(test_dir)
print(f"Found {len(test_folders)} folders to process")

# Process each folder
success_count = 0
fail_count = 0

for folder in tqdm(test_folders, desc="Processing folders"):
    # Look for inverse_annotation.json in the folder
    inverse_file = os.path.join(folder, "inverse_annotation.json")
    print(inverse_file)
    if os.path.exists(inverse_file):
        result = update_inverse_file(inverse_file)
        if result:
            success_count += 1
        else:
            fail_count += 1
    else:
        print(f"No inverse_annotation.json found in {folder}")
        fail_count += 1
    
    # if success_count == 2: 
    #     break
print(f"\nProcessing complete!")
print(f"Successfully updated: {success_count} folders")
print(f"Failed or no changes needed: {fail_count} folders")

Found 4054 folders to process


Processing folders:   0%|          | 0/4054 [00:00<?, ?it/s]

queries_dataset/merged_balanced/inverse_search/test\0\inverse_annotation.json
queries_dataset/merged_balanced/inverse_search/test\1\inverse_annotation.json
queries_dataset/merged_balanced/inverse_search/test\10\inverse_annotation.json
queries_dataset/merged_balanced/inverse_search/test\100\inverse_annotation.json


Processing folders:   0%|          | 4/4054 [00:01<30:13,  2.23it/s]

{'entities': ['Delicious Deli', 'Car', 'Traffic', 'Luxury car', 'Motor vehicle', 'Sport utility vehicle', 'Van', 'Street', 'Compact car', 'Pedestrian', 'Parking', 'Transport', 'Road', 'Clydebank', 'asphalt'], 'all_matched_captions': [{'page_link': 'https://www.bbc.com/news/uk-scotland-glasgow-west-35892821', 'image_link': 'https://ichef.bbci.co.uk/news/640/cpsprodpb/AB74/production/_88929834_paige_deligood.jpg', 'html_path': 'queries_dataset\\merged_balanced\\inverse_search\\test\\100\\0.txt', 'title': 'Paige Doherty death: Man arrested in Clydebank area - BBC News', 'content': 'Paige Doherty was last seen near the Delicious Deli in Clydebank on Saturday morning\n\nA man has been arrested in connection with the death of 15-year-old Paige Doherty whose body was found in a wooded area in Clydebank.\n\nThe teenager was last seen near a deli on Saturday morning after setting off for her part-time hairdressing job.\n\nHer body was found just off Great Western Road at lunchtime on Monday.\n\

...

f:\Thesis\DKH-Thesis\venv\lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Processing folders:   1%|          | 43/4054 [02:31<7:34:19,  6.80s/it]

{'entities': ['Cherry blossom', 'National Cherry Blossom Festival', 'Sakura', 'Sakura', 'Image', 'Blossom', 'Sweet Cherry', 'Painting', 'Spring', 'Cherry', 'Cherry blossom'], 'all_matched_captions': [{'page_link': 'https://lat.rtrs.tv/foto/foto.php?id=32727&glry=2683', 'image_link': 'https://lat.rtrs.tv/_FOTO/glrz/0327/032727.jpg', 'html_path': 'queries_dataset\\merged_balanced\\inverse_search\\test\\1035\\4.txt', 'title': 'RTRS', 'content': '', 'caption': {'alt_node': 'Јapanska trešnja u cvatu (Foto:Profimedia.rs)'}, 'image_path': 'queries_dataset/merged_balanced/inverse_search/test\\1035\\4.jpg'}, {'page_link': 'https://paintingvalley.com/famous-cherry-blossom-tree-painting', 'image_link': 'https://paintingvalley.com/images/famous-cherry-blossom-tree-painting-13.jpg', 'html_path': 'queries_dataset\\merged_balanced\\inverse_search\\test\\1035\\1.txt', 'title': 'Famous Cherry Blossom Tree Painting at PaintingValley.com | Explore collection of Famous Cherry Blossom Tree Painting', 'cont

Processing folders:   1%|          | 43/4054 [02:45<4:16:34,  3.84s/it]


KeyboardInterrupt: 

: 